# Schedule recordings using saved orbital data

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [3]:
library(yaml)

In [4]:
library(assertthat)


Attaching package: ‘assertthat’

The following object is masked from ‘package:tibble’:

    has_name



In [5]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-28 08:56:52 GMT"

In [6]:
assert_that(file.exists("satellite_data_df.rds"),
            msg = "No satellite data files found on disk. Have you run `Satellite_orbital_data`?")

[1] TRUE

In [43]:
satellite_data_df <- readRDS("satellite_data_df.rds")

In [44]:
assert_that(file.exists("calibration.yaml"),
            msg = "No dongle calibration data on disk. Have you run `Calibrate_receiver`?")

[1] TRUE

In [45]:
calibration_data <- read_yaml("calibration.yaml")

In [46]:
calibration_data

$clock_offset
[1] -0.605

In [47]:
clock_offset <- calibration_data$clock_offset

## Convert to satellite passes each day

Get the current time in UTC

In [48]:
current_time_utc <- Sys.time() %>% .POSIXct("GMT")

Make sure we have a future satellite pass in the data, then filter out any past passes.

In [49]:
assert_that(nrow(satellite_data_df %>% filter(startUTC > current_time_utc)) > 0,
            msg = "No future satellite orbital data available")

[1] TRUE

In [50]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

## Format systemd

For systemd, using `systemd-run` rather than the usual template based form since these are one-off non-recurring commands. The argument `--user` runs the command as the user who scheduled it. `--unit` gives the job a name. The argument `--on-calendar` schedules the start and needs to be one minute before the satellite pass to give systemd and mlrpt time to start up. The format is yyyy-mm-dd HH:MM in **local time**. Documentation at https://www.freedesktop.org/software/systemd/man/systemd-run.html

## Format `rtl_fm`

rtl_fm -M raw -s 140000 -f 137.9M -E dc -g <gain> -p <ppm>

http://kmkeen.com/rtl-demod-guide/

`timeout $duration rtl_fm $biast -f $freq -s $sample -g $dongleGain -F 9 -A fast -E offset -p $dongleShift $recdir/$fileNameCore.raw | tee -a $logFile`

rtl_fm will record indefinitely. Use `timeout` to set a recording duration. **Append `m` to the time for minutes.** Seconds is the default

`-M raw` Raw output, 2x16 bit IQ pairs

`-s` Sample rate, width of recorded signal - should include few kHz for doppler shift.
sample_rate='48000'

`-g` Dongle gain. Run `rtl_test` to get supported gain levels. 49.6 seems to be the highest supported by rtl-sdr.
dongleGain='49.6'

`-F` fir_size (default: off) enables low-leakage downsample filterEnable a higher quality downsampling FIR than the default boxcar filter. `-F 0` is okay to use while `-F 9` is still a work in progress.
FIR=9

`-A` Choose how arctan is computed. Options select between the standard (floating) lib *std*, a fast polynomial integer approximation *fast* and a precomputed look-up-table *lut*
arctan=fast

`-p` Error. Correct for the parts-per-million error in the crystal. Run `Calibrate_receiver.ipynb` to calculate.
dongleshift=clock_offset

`-E` Use multiple -E options to enable multiple features.
    1) `-E dc` dc blocking filter
    2) `-E deemp` de-emphasis filter for WBFM
    3) `-E direct` direct sampling mode
    4) `-E offset` offset tuning mode (E4000 only)
    
`-T` To turn bias-t power on (i.e., for LNA) use the `-T` flag 

In [ ]:
paste0('timeout ', df$duration,'m rtl_fm -f ', df$freq, ' -s ', $sample, ' -g ', $dongleGain,
       -F 9 -A fast -E offset -p $dongleShift 

In [40]:
setSatelliteRecording <- function(df) {
    df <- df %>% mutate(bash_command = 
        case_when(satellite == "METEOR-M2" ~ 
            paste0('systemd-run --user --unit=\"mlrpt_', df$startDate, "_", df$localStartTime,
            '\" --on-calendar=\"', df$startDate," ", df$localStartTime,
            '\" /usr/local/bin/mlrpt -s ', df$startTime,"-", df$endTime,
            ' -t ', df$duration),
            satellite == "METEOR-M2-2" ~ paste0('systemd-run --user --unit=\"mlrpt_', df$startDate, "_", df$localStartTime,
            '\" --on-calendar=\"', df$startDate," ", df$localStartTime,
            '\" /usr/local/bin/mlrpt -s ', df$startTime,"-", df$endTime,
            ' -t ', df$duration),
            satellite == "NOAA-15" ~ paste0('systemd-run --user --unit=\"mlrpt_', df$startDate, "_", df$localStartTime,
            '\" --on-calendar=\"', df$startDate," ", df$localStartTime,
            '\" /usr/local/bin/mlrpt -s ', df$startTime,"-", df$endTime,
            ' -t ', df$duration),
            satellite == "NOAA-18" ~ paste0('systemd-run --user --unit=\"mlrpt_', df$startDate, "_", df$localStartTime,
            '\" --on-calendar=\"', df$startDate," ", df$localStartTime,
            '\" /usr/local/bin/mlrpt -s ', df$startTime,"-", df$endTime,
            ' -t ', df$duration),
            satellite == "NOAA-19" ~ paste0('systemd-run --user --unit=\"mlrpt_', df$startDate, "_", df$localStartTime,
            '\" --on-calendar=\"', df$startDate," ", df$localStartTime,
            '\" /usr/local/bin/mlrpt -s ', df$startTime,"-", df$endTime,
            ' -t ', df$duration)))
    return(df)
}

In [39]:
paste0('systemd-run --user --unit=\"mlrpt_', satellite_data_df$startDate, "_", satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime, 
       satellite_data_df$bash_command)

[1] "systemd-run --user --unit=\"mlrpt_2019-09-28_15:21\" --on-calendar=\"2019-09-28 15:21\" /usr/local/bin/mlrpt -s 1322-1337 -t 16"
  [2] "systemd-run --user --unit=\"mlrpt_2019-09-28_19:05\" --on-calendar=\"2019-09-28 19:05\" /usr/local/bin/mlrpt -s 1706-1721 -t 16"
  [3] "systemd-run --user --unit=\"mlrpt_2019-09-28_20:14\" --on-calendar=\"2019-09-28 20:14\" /usr/local/bin/mlrpt -s 1815-1830 -t 16"
  [4] "systemd-run --user --unit=\"mlrpt_2019-09-28_20:33\" --on-calendar=\"2019-09-28 20:33\" /usr/local/bin/mlrpt -s 1834-1850 -t 16"
  [5] "systemd-run --user --unit=\"mlrpt_2019-09-28_20:33\" --on-calendar=\"2019-09-28 20:33\" /usr/local/bin/mlrpt -s 1834-1850 -t 16"
  [6] "systemd-run --user --unit=\"mlrpt_2019-09-29_02:36\" --on-calendar=\"2019-09-29 02:36\" /usr/local/bin/mlrpt -s 0037-0052 -t 16"
  [7] "systemd-run --user --unit=\"mlrpt_2019-09-29_06:18\" --on-calendar=\"2019-09-29 06:18\" /usr/local/bin/mlrpt -s 0419-0435 -t 16"
  [8] "systemd-run --user --unit=\"mlrpt_2019-09-29_07:30\" --on-calendar=\"2019-09-29 07:30\" /usr/local/bin/mlrpt -s 0531-0546 -t 15"
  [9] "systemd-run --user --unit=\"mlrpt_2019-09-29_07:54\" --on-calendar=\"2019-09-29 07:54\" /usr/local/bin/mlrpt -s 0555-0610 -t 16"
 [10] "systemd-run --user --unit=\"mlrpt_2019-09-29_07:54\" --on-calendar=\"2019-09-29 07:54\" /usr/local/bin/mlrpt -s 0555-0610 -t 16"
 [11] "systemd-run --user --unit=\"mlrpt_2019-09-29_14:59\" --on-calendar=\"2019-09-29 14:59\" /usr/local/bin/mlrpt -s 1300-1315 -t 16"
 [12] "systemd-run --user --unit=\"mlrpt_2019-09-29_18:40\" --on-calendar=\"2019-09-29 18:40\" /usr/local/bin/mlrpt -s 1641-1656 -t 15"
 [13] "systemd-run --user --unit=\"mlrpt_2019-09-29_19:54\" --on-calendar=\"2019-09-29 19:54\" /usr/local/bin/mlrpt -s 1755-1810 -t 15"
 [14] "systemd-run --user --unit=\"mlrpt_2019-09-29_20:22\" --on-calendar=\"2019-09-29 20:22\" /usr/local/bin/mlrpt -s 1823-1838 -t 15"
 [15] "systemd-run --user --unit=\"mlrpt_2019-09-29_20:22\" --on-calendar=\"2019-09-29 20:22\" /usr/local/bin/mlrpt -s 1823-1838 -t 15"
 [16] "systemd-run --user --unit=\"mlrpt_2019-09-29_21:35\" --on-calendar=\"2019-09-29 21:35\" /usr/local/bin/mlrpt -s 1936-1949 -t 14"
 [17] "systemd-run --user --unit=\"mlrpt_2019-09-29_22:03\" --on-calendar=\"2019-09-29 22:03\" /usr/local/bin/mlrpt -s 2004-2018 -t 14"
 [18] "systemd-run --user --unit=\"mlrpt_2019-09-29_22:03\" --on-calendar=\"2019-09-29 22:03\" /usr/local/bin/mlrpt -s 2004-2018 -t 14"
 [19] "systemd-run --user --unit=\"mlrpt_2019-09-30_02:14\" --on-calendar=\"2019-09-30 02:14\" /usr/local/bin/mlrpt -s 0015-0030 -t 15"
 [20] "systemd-run --user --unit=\"mlrpt_2019-09-30_05:54\" --on-calendar=\"2019-09-30 05:54\" /usr/local/bin/mlrpt -s 0355-0409 -t 15"
 [21] "systemd-run --user --unit=\"mlrpt_2019-09-30_07:11\" --on-calendar=\"2019-09-30 07:11\" /usr/local/bin/mlrpt -s 0512-0526 -t 15"
 [22] "systemd-run --user --unit=\"mlrpt_2019-09-30_07:34\" --on-calendar=\"2019-09-30 07:34\" /usr/local/bin/mlrpt -s 0535-0549 -t 14"
 [23] "systemd-run --user --unit=\"mlrpt_2019-09-30_07:42\" --on-calendar=\"2019-09-30 07:42\" /usr/local/bin/mlrpt -s 0543-0558 -t 15"
 [24] "systemd-run --user --unit=\"mlrpt_2019-09-30_07:42\" --on-calendar=\"2019-09-30 07:42\" /usr/local/bin/mlrpt -s 0543-0558 -t 15"
 [25] "systemd-run --user --unit=\"mlrpt_2019-09-30_08:51\" --on-calendar=\"2019-09-30 08:51\" /usr/local/bin/mlrpt -s 0652-0706 -t 15"
 [26] "systemd-run --user --unit=\"mlrpt_2019-09-30_09:23\" --on-calendar=\"2019-09-30 09:23\" /usr/local/bin/mlrpt -s 0724-0738 -t 14"
 [27] "systemd-run --user --unit=\"mlrpt_2019-09-30_09:23\" --on-calendar=\"2019-09-30 09:23\" /usr/local/bin/mlrpt -s 0724-0738 -t 14"
 [28] "systemd-run --user --unit=\"mlrpt_2019-09-30_14:36\" --on-calendar=\"2019-09-30 14:36\" /usr/local/bin/mlrpt -s 1237-1252 -t 15"
 [29] "systemd-run --user --unit=\"mlrpt_2019-09-30_18:15\" --on-calendar=\"2019-09-30 18:15\" /usr/local/bin/mlrpt -s 1616-1630 -t 14"
 [30] "systemd-run --user --unit=\"mlrpt_2019-09-30_19:35\

In [42]:
mutate(satellite_data_df, bash_command = ifelse(satellite == "METEOR-M2" ,
    paste0('systemd-run --user --unit=\"mlrpt_', startDate, "_", localStartTime,
    '\" --on-calendar=\"', startDate," ", localStartTime,
    '\" /usr/local/bin/mlrpt -s ', startTime,"-", endTime,
    ' -t ', duration), bash_command))

satellite,frequency,norad_id,priority,startAz,startAzCompass,startUTC,maxAz,maxAzCompass,maxEl,⋯,endAz,endAzCompass,endUTC,startDate,startTime,endDate,endTime,localStartTime,duration,bash_command
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dttm>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
METEOR-M2-2,137100000,44387,5,170.89,S,2019-09-28 13:22:40,258.03,W,71.22,⋯,340.44,NNW,2019-09-28 13:37:55,2019-09-28,1322,2019-09-28,1337,15:21,16,NA
NOAA-15,137620000,25338,3,165.58,S,2019-09-28 17:06:15,72.18,ENE,76.15,⋯,349.88,N,2019-09-28 17:21:20,2019-09-28,1706,2019-09-28,1721,19:05,16,NA
METEOR-M2,137100000,40069,4,161.76,SSE,2019-09-28 18:15:30,79.95,E,54.56,⋯,357.59,N,2019-09-28 18:30:40,2019-09-28,1815,2019-09-28,1830,20:14,16,"systemd-run --user --unit=""mlrpt_2019-09-28_20:14"" --on-calendar=""2019-09-28 20:14"" /usr/local/bin/mlrpt -s 1815-1830 -t 16"
NOAA-18,137912500,28654,2,158.92,SSE,2019-09-28 18:34:55,81.24,E,46.09,⋯,0.89,N,2019-09-28 18:50:15,2019-09-28,1834,2019-09-28,1850,20:33,16,NA
NOAA-19,137100000,28654,1,158.92,SSE,2019-09-28 18:34:55,81.24,E,46.09,⋯,0.89,N,2019-09-28 18:50:15,2019-09-28,1834,2019-09-28,1850,20:33,16,NA
METEOR-M2-2,137100000,44387,5,11.75,N,2019-09-29 00:37:30,275.89,W,82.26,⋯,193.31,S,2019-09-29 00:52:45,2019-09-29,0037,2019-09-29,0052,02:36,16,NA
NOAA-15,137620000,25338,3,21.69,NNE,2019-09-29 04:19:55,107.86,ESE,64.99,⋯,188.16,S,2019-09-29 04:35:00,2019-09-29,0419,2019-09-29,0435,06:18,16,NA
METEOR-M2,137100000,40069,4,28.07,NNE,2019-09-29 05:31:45,106.17,ESE,48.70,⋯,184.41,S,2019-09-29 05:46:45,2019-09-29,0531,2019-09-29,0546,07:30,15,"systemd-run --user --unit=""mlrpt_2019-09-29_07:30"" --on-calendar=""2019-09-29 07:30"" /usr/local/bin/mlrpt -s 0531-0546 -t 15"
NOAA-18,137912500,28654,2,29.34,NNE,2019-09-29 05:55:05,107.83,ESE,47.77,⋯,184.82,S,2019-09-29 06:10:15,2019-09-29,0555,2019-09-29,0610,07:54,16,NA


Should try medet https://www.reddit.com/r/RTLSDR/comments/cvcqg9/i_finally_added_meteor_m22_support_to_medet/ https://github.com/artlav/meteor_decoder https://github.com/dbdexter-dev/meteor_demod https://www.reddit.com/r/RTLSDR/comments/abn29d/automatic_meteor_m2_reception_on_linux_using_rtl/  instead

## Format systemd / rtl_fm commands 

http://kmkeen.com/rtl-demod-guide/

`timeout $duration rtl_fm $biast -f $freq -s $sample -g $dongleGain -F 9 -A fast -E offset -p $dongleShift $recdir/$fileNameCore.raw | tee -a $logFile`

-For `timeout`, append `m` to the time for minutes. Seconds is the default

-Whether to turn bias-t power on (i.e., for LNA). See: `rtl_fm --help | grep "-T"`
biast="" 

-Sample rate, width of recorded signal - should include few kHz for doppler shift.
sample='48000'

-Dongle gain. Run `rtl_test` to get supported gain levels. 49.6 seems to be the highest supported by rtl-sdr.
dongleGain='49.6'

-`-F` fir_size (default: off) enables low-leakage downsample filterEnable a higher quality downsampling FIR than the default boxcar filter. `-F 0` is okay to use while `-F 9` is still a work in progress.
FIR=9

-`-A` Choose how arctan is computed. Options select between the standard (floating) lib *std*, a fast polynomial integer approximation *fast* and a precomputed look-up-table *lut*
arctan=fast

dongleshift

- `-E`
    1) `-E dc` dc blocking filter
    2) `-E deemp` de-emphasis filter for WBFM
    3) `-E direct` direct sampling mode
    4) `-E offset` offset tuning mode (E4000 only)
    Use multiple -E option to enable multiple features.


-Sample rate of the wav file. Shouldn't be changed
wavrate='11025'

-Dongle index, is there any rtl_fm allowing passing serial of dongle?
dongleIndex='0'

-enchancements to apply to the pocessed images. See wxtoimg manual for available options
enchancements=('MCIR-precip' 'HVC' 'MSA' 'therm' 'HVCT-precip' 'NO')

-resize images to the given size to avoid growing of the repository; in px, otherwise, comment out the line
resizeimageto=1024

In [15]:
paste0('systemd-run --user --unit=\"rtl_fm_', satellite_data_df$startDate, "_", satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime,
       '\" /usr/bin/timeout ', satellite_data_df$duration, 'm', ' /usr/local/bin/rtl_fm -f ', satellite_data_df$frequency,
       ' -s 48000 -g 49.6 -F 9 -A fast -E offset -p ', clock_offset, 
       ' ', satellite_data_df$startDate, "_", satellite_data_df$localStartTime, '.raw')

[1] "systemd-run --user --unit=\"rtl_fm_2019-09-22_21:44\" --on-calendar=\"2019-09-22 21:44\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137912500 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-22_21:44.raw"
  [2] "systemd-run --user --unit=\"rtl_fm_2019-09-22_21:44\" --on-calendar=\"2019-09-22 21:44\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-22_21:44.raw"
  [3] "systemd-run --user --unit=\"rtl_fm_2019-09-22_01:32\" --on-calendar=\"2019-09-22 01:32\" /usr/bin/timeout 14m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-22_01:32.raw"
  [4] "systemd-run --user --unit=\"rtl_fm_2019-09-23_03:12\" --on-calendar=\"2019-09-23 03:12\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_03:12.raw"
  [5] "systemd-run --user --unit=\"rtl_fm_2019-09-23_05:31\" --on-calendar=\"2019-09-23 05:31\" /usr/bin/timeout 14m /usr/local/bin/rtl_fm -f 137620000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_05:31.raw"
  [6] "systemd-run --user --unit=\"rtl_fm_2019-09-23_07:09\" --on-calendar=\"2019-09-23 07:09\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137620000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_07:09.raw"
  [7] "systemd-run --user --unit=\"rtl_fm_2019-09-23_07:24\" --on-calendar=\"2019-09-23 07:24\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137912500 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_07:24.raw"
  [8] "systemd-run --user --unit=\"rtl_fm_2019-09-23_07:24\" --on-calendar=\"2019-09-23 07:24\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_07:24.raw"
  [9] "systemd-run --user --unit=\"rtl_fm_2019-09-23_07:49\" --on-calendar=\"2019-09-23 07:49\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_07:49.raw"
 [10] "systemd-run --user --unit=\"rtl_fm_2019-09-23_09:04\" --on-calendar=\"2019-09-23 09:04\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137912500 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_09:04.raw"
 [11] "systemd-run --user --unit=\"rtl_fm_2019-09-23_09:04\" --on-calendar=\"2019-09-23 09:04\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_09:04.raw"
 [12] "systemd-run --user --unit=\"rtl_fm_2019-09-23_15:34\" --on-calendar=\"2019-09-23 15:34\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_15:34.raw"
 [13] "systemd-run --user --unit=\"rtl_fm_2019-09-23_19:30\" --on-calendar=\"2019-09-23 19:30\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137620000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_19:30.raw"
 [14] "systemd-run --user --unit=\"rtl_fm_2019-09-23_20:13\" --on-calendar=\"2019-09-23 20:13\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_20:13.raw"
 [15] "systemd-run --user --unit=\"rtl_fm_2019-09-23_21:32\" --on-calendar=\"2019-09-23 21:32\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137912500 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_21:32.raw"
 [16] "systemd-run --user --unit=\"rtl_fm_2019-09-23_21:32\" --on-calendar=\"2019-09-23 21:32\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_21:32.raw"
 [17] "systemd-run --user --unit=\"rtl_fm_2019-09-24_02:49\" --on-calendar=\"2019-09-24 02:49\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-24_02:49.raw"
 [18] "systemd-run --user --unit=\"rtl_fm_2019-09-24_06:44\" --on-calendar=\"2019-09-24 06:44\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137620000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p